# Independnet Study Week 3 - Clustering / Text Processing

## Preperation

In [42]:
import numpy as np
import igraph
import pickle
import requests
import boilerpipe
import opengraph
import pprint

%pylab inline

Populating the interactive namespace from numpy and matplotlib


## Part 1 - Heirarchical Clustering

The only algo in the readings for clustering is (agglomerative) heirarchical clustering, which I attempt to implement here.

In [ ]:
#TODO

## Part 2  - Text Processing

Continuing from last week, I extend the textual analysis to the articles linked to tweets from the think-tank accounts. Lets start off importing statuses from pickle object, listing out our accounts and extracting all the urls we can get.

In [68]:
# Open content and accounts from previous week
allStatuses = pickle.load( open( "../Week2/allStatuses", "rb" ) )
accounts = allStatuses.keys()

# Extract all the links from the crawled tweets
links = { }
for account in accounts:
    links[account] = [url.expanded_url  for status in allStatuses[account] for url in status.urls ]

Lets look at how boilerpipe works for a single article:

In [110]:
# Boiler pipe for a single article example
from boilerpipe.extract import Extractor
extractor = Extractor(extractor='ArticleExtractor', url=links['AccuracyInMedia'][0])
extracted_text_default = extractor.getText()
print extracted_text_default

Home Faculty Lounge Rachel Dolezal Let Go at Eastern Washington University
Rachel Dolezal Let Go at Eastern Washington University
June 25, 2015
,  Spencer Irvine, Leave a comment
The controversial adjunct professor and former head of the Spokane, Washington NAACP chapter will not be retained, in all likelihood. But, her past is more head-scratching than her recent comments on her being African-American.
Book Review
Have We Lost the Cultural War?
Paul Kengor’s new book, Takedown: From Communists to Progressives, How the Left Has Sabotaged Family and Marriage, provides a detailed explanation of why, according to a new Gallup poll, “Americans are more likely now than…
, Spencer Irvine , No Comment
Coming to a university near you
Before you find him on offer as a university speaker or course, you may want to read the meticulously documented story of Cop killer Mumia Abu-Jamal by former Accuracy in Academia executive director Dan Flynn.
© Accuracy In Academia
Receive our FREE email newslett

As you can see, the results are okay. There are different kinds of extractors, but this one seems to do well enough. Boilerpipe does better with articles from FAIR. Let us look at how we could use opengraph to extract information (if boilerpipe doesn't work well)

In [76]:
for url in links["AccuracyInMedia"][:5]:
    og =  opengraph.OpenGraph(url=url, scrape=True)
    print "%s. %s" % (og.title, og.description)
    print

Rachel Dolezal Let Go at Eastern Washington University. The controversial adjunct professor and former head of the Spokane, Washington NAACP chapter will not be retained, in all likelihood. But, her past is more head-scratching than her recent comments ...

Supreme Court Upholds ObamaCare, Again, in King v. Burwell  . Yup, by a 6-3 vote and the majority written by Chief Justice Roberts, ObamaCare’s illegal subsidies are ruled legal.

IRS shelled out $18.8 Million to Contractors who didn’t pay Back Taxes  . Way to protect American taxpayer dollars, IRS. “The IRS granted 57 contracts worth $18.8 million to corporations with unpaid back taxes from 2012 to 2013, according to a March 26 Treasury Inspector General for Tax Administration (TIGTA) report.”

Study finds less than 5% of Colleges Protect 1st Amendment Rights  . This isn’t shocking news as colleges like to impose speech codes and free speech zones on their campuses: “A new study says that less than 5 percent of colleges and univers

This looks a little better, though it is not much more data. Lets use it and crawl the links for all accounts using BoilerPipe and OpenGraph as a backup in case of an error:

In [104]:
content = {}
LIMIT = 50

# Try to open content from pickle file
try:
    content = pickle.load(open("crawledContent.pkl", "rb"))
except Exception as e:
    # Otherwise, loop thru all acounts
    for account in accounts:
        # Init account to have no content
        content[account] = {}
        print "Starting account %s" % account

        # Loop thru all links for the account
        for url in links[account][:LIMIT]:
            # Attempt to use boilerpipe's article extractor on url
            try:
                print " -- extracting %s with boilerpipe" % url
                extractor = Extractor(extractor='ArticleExtractor', url=url)
                content[account][url] = extractor.getText()
            except Exception as e:
                # If there is any issue, try using OpenGraph metadata
                try:
                    print " -- failed; extracting %s with og" % url
                    og =  opengraph.OpenGraph(url=url, scrape=True)
                    content[account][url] = "%s. %s" % (og.title, og.description)
                except Exception as e:
                    pass
    
    # Save results
    pickle.dump(content, open("crawledContent.pkl", "wb"))

Starting account ips_dc
 -- extracting http://www.ips-dc.org/?p=38436 with boilerpipe
 -- extracting http://fpif.org/our-refugee-world with boilerpipe
 -- extracting http://www.ips-dc.org/francis-v-washington/ with boilerpipe
 -- extracting http://www.ips-dc.org/?p=38416 with boilerpipe
 -- extracting http://bit.ly/1L3dMtX with boilerpipe
 -- extracting http://bit.ly/1BAKYqo with boilerpipe
 -- extracting http://www.ips-dc.org/?p=38414 with boilerpipe
 -- extracting http://bit.ly/1IyIUfP with boilerpipe
 -- extracting http://www.ips-dc.org/?p=38412 with boilerpipe
 -- extracting http://www.ips-dc.org/?p=38410 with boilerpipe
 -- extracting http://www.ips-dc.org/?p=38410 with boilerpipe
 -- extracting http://bit.ly/1LiZ565 with boilerpipe
 -- extracting http://www.ips-dc.org/?p=38399 with boilerpipe
 -- extracting http://www.ips-dc.org/hacked/ with boilerpipe
 -- extracting http://www.ips-dc.org/?p=38262 with boilerpipe
 -- extracting http://ips-dc.org/?p=38392 with boilerpipe
 -- extra

Now that we have crawled some data, lets process it to find some collocations (with bigrams)

In [98]:
import string

import nltk
from nltk.collocations import *
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

def preProcess(text):
    text = text.lower()
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(text)
    filtered_words = [w for w in tokens if not w in stopwords.words('english') and not w.isdigit()]
    return " ".join(filtered_words)

def displayBigrams(contentDict, threshold=5):
    tokens = nltk.wordpunct_tokenize(preProcess("".join([ contentDict[url] for url in contentDict])))
    bigram_measures = nltk.collocations.BigramAssocMeasures()
    finder = BigramCollocationFinder.from_words(tokens)
    finder.apply_freq_filter(threshold)
    scored = finder.score_ngrams(bigram_measures.raw_freq)
    return sorted(bigram for bigram, score in scored)

Lets create a list of bigrams from AccuracyInMedia:

In [105]:
displayBigrams(content['AccuracyInMedia'])

[(u'abortion', u'clinics'),
 (u'account', u'log'),
 (u'account', u'sign'),
 (u'accuracy', u'academia'),
 (u'aim', u'running'),
 (u'author', u'spencer'),
 (u'blog', u'target'),
 (u'brigham', u'young'),
 (u'burwell', u'spencer'),
 (u'burwell', u'supreme'),
 (u'care', u'get'),
 (u'chief', u'justice'),
 (u'clinton', u'campaign'),
 (u'close', u'sign'),
 (u'close', u'two'),
 (u'codes', u'country'),
 (u'confederate', u'flag'),
 (u'country', u'home'),
 (u'court', u'upholds'),
 (u'currently', u'works'),
 (u'deal', u'iran'),
 (u'e', u'mails'),
 (u'email', u'address'),
 (u'extracurricular', u'activities'),
 (u'get', u'updates'),
 (u'graduated', u'brigham'),
 (u'happen', u'account'),
 (u'hillary', u'clinton'),
 (u'home', u'blog'),
 (u'illegal', u'subsidies'),
 (u'international', u'relations'),
 (u'irvine', u'graduated'),
 (u'irvine', u'june'),
 (u'irvine', u'spencer'),
 (u'june', u'yup'),
 (u'justice', u'roberts'),
 (u'king', u'v'),
 (u'legal', u'share'),
 (u'log', u'close'),
 (u'majority', u'writ

Comapring it with fairmediawatch:

In [106]:
displayBigrams(content['fairmediawatch'])

[(u'accuracy', u'reporting'),
 (u'act', u'violence'),
 (u'adam', u'johnson'),
 (u'american', u'life'),
 (u'anti', u'privacy'),
 (u'baltimore', u'police'),
 (u'baltimore', u'uprising'),
 (u'barack', u'obama'),
 (u'bernie', u'sanders'),
 (u'black', u'people'),
 (u'black', u'white'),
 (u'blog', u'zhome'),
 (u'boston', u'marathon'),
 (u'california', u'drought'),
 (u'charleston', u'massacre'),
 (u'charter', u'schools'),
 (u'com', u'public'),
 (u'communication', u'effective'),
 (u'corporate', u'media'),
 (u'coverage', u'women'),
 (u'daily', u'news'),
 (u'data', u'collection'),
 (u'de', u'blasio'),
 (u'dean', u'baker'),
 (u'democratic', u'nomination'),
 (u'dylann', u'roof'),
 (u'ed', u'snowden'),
 (u'editor', u'fair'),
 (u'editor', u'margaret'),
 (u'effective', u'related'),
 (u'even', u'though'),
 (u'face', u'nation'),
 (u'facebook', u'likes'),
 (u'fair', u'blog'),
 (u'fair', u'org'),
 (u'fairness', u'accuracy'),
 (u'far', u'left'),
 (u'far', u'right'),
 (u'filed', u'blog'),
 (u'fox', u'news'